In [11]:
# Import libraries
import gradio as gr
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [15]:
import gradio as gr
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib  # For saving and loading models

# Load data from CSV
data = pd.read_csv('fraudTest.csv')

# Preprocessing data
# Assuming 'category', 'merchant' are categorical and need encoding
merchant_encoder = LabelEncoder()
category_encoder = LabelEncoder()

data['category'] = category_encoder.fit_transform(data['category'])
data['merchant'] = merchant_encoder.fit_transform(data['merchant'])

# Save the encoders for later use in prediction
joblib.dump(merchant_encoder, 'merchant_encoder.pkl')
joblib.dump(category_encoder, 'category_encoder.pkl')

# Select features and target
X = data[['amt', 'merchant', 'category']]
y = data['is_fraud']  # Assuming there is a column 'is_fraud' that indicates if the transaction was fraudulent

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Define the prediction function for Gradio
def predict_fraud(amount, merchant, category):
    # Load the saved LabelEncoders
    merchant_encoder = joblib.load('merchant_encoder.pkl')
    category_encoder = joblib.load('category_encoder.pkl')
    
    # Encode the input values
    try:
        merchant_encoded = merchant_encoder.transform([merchant])[0]
        category_encoded = category_encoder.transform([category])[0]
    except ValueError:
        return "Error: Merchant or Category not recognized. Please check your input."

    # Make prediction
    prediction = model.predict([[amount, merchant_encoded, category_encoded]])
    return 'Fraudulent' if prediction[0] == 1 else 'Non-Fraudulent'

# Setup the Gradio interface
iface = gr.Interface(fn=predict_fraud,
                     inputs=["number", "text", "text"],
                     outputs="text",
                     title="Credit Card Fraud Detection",
                     description="Enter transaction details to predict if they are fraudulent.")
iface.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


/opt/homebrew/Caskroom/miniconda/base/envs/fraud-detection-env/lib/python3.8/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/fraud-detection-env/lib/python3.8/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/fraud-detection-env/lib/python3.8/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/fraud-detection-env/lib/python3.8/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/fraud-detecti